# Note:  This requires a set of large instance type due to the amount of data.

In [1]:
!pip install -q boto3

tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
tensorflow 2.0.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.14.3 which is incompatible.
awscli 1.18.20 has requirement botocore==1.15.20, but you'll have botocore 1.15.23 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Specify the S3 Location of the Features

In [5]:
prefix_train = '{}/raw-labeled-split-balanced-header-train'.format(spark_processing_job_s3_output_prefix)
prefix_validation = '{}/raw-labeled-split-balanced-header-validation'.format(spark_processing_job_s3_output_prefix)
prefix_test = '{}/raw-labeled-split-balanced-header-test'.format(spark_processing_job_s3_output_prefix)

path_train = './{}'.format(prefix_train)
path_validation = './{}'.format(prefix_validation)
path_test = './{}'.format(prefix_test)

train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

s3_input_train_data = sagemaker.s3_input(s3_data=train_s3_uri, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=validation_s3_uri, content_type='text/csv')
s3_input_test_data = sagemaker.s3_input(s3_data=test_s3_uri, content_type='text/csv')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-21-01-02-23/raw-labeled-split-balanced-header-train', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-21-01-02-23/raw-labeled-split-balanced-header-validation', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-21-01-02-23/raw-labeled-split-balanced-header-test', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


In [6]:
!cat src_bert/xgboost_reviews_bert.py

import os
import argparse
import csv
import pickle as pkl
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
import sklearn
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
import re
import xgboost as xgb
from xgboost import XGBClassifier
import glob
from simpletransformers.classification import ClassificationModel

# Note:  header=None
def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    labels = data.iloc[:,0]
    features = data.drop(data.columns[0], axis=1)
    
    if header==None:
        # Adjust the column names after dropped the 0th column above
        # New column names are 0 (inclusive) to len(features.columns) (exclusive)
        new_column_names = list(range(0, len(features.columns)))
        features.columns = new_column_names

    return f

In [13]:
from sagemaker.pytorch import PyTorch

model_output_path = 's3://{}/models/amazon-reviews/script-mode/training-runs'.format(bucket)

bert_estimator = PyTorch(entry_point='xgboost_reviews_bert.py',
                           source_dir='src_bert',
                           role=role,
                           train_instance_count=1, 
                           train_instance_type='ml.c5.9xlarge',
                           py_version='py3',
                           framework_version='1.4.0',
                           output_path=model_output_path,
                           hyperparameters={'model_type':'bert',
                                            'model_name': 'bert-base-cased'},
                           enable_cloudwatch_metrics=True)

### Train the model

In [14]:
bert_estimator.fit(inputs={'train': s3_input_train_data, 
                           'validation': s3_input_validation_data,}, 
                   wait=False) 

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-21-01-02-23/raw-labeled-split-balanced-header-train" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881" has "s3:ListBucket" permissions on bucket "sagemaker-us-east-1-835319576252".

In [ ]:
training_job_name = bert_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

In [ ]:
from sagemaker.pytorch import PyTorch

bert_estimator = PyTorch.attach(training_job_name=training_job_name)